In [1]:
# __enter__, __exit__
with open('cafe.txt') as fp:
    src = fp.read(30)

len(src)

4

In [2]:
fp

<_io.TextIOWrapper name='cafe.txt' mode='r' encoding='UTF-8'>

In [3]:
fp.closed, fp.encoding

(True, 'UTF-8')

In [5]:
fp.read(4) # I/O error __exit__ was called after the with block

ValueError: I/O operation on closed file.

In [10]:
import sys

class LookingGlass:

    def __enter__(self):
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return 'ICECREAM'

    def reverse_write(self, text):
        self.original_write(text[::-1])

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print('Please DO NOT divide by zero!')
            return True

# usually this would be imported method:
# from filename import LookingGlass
with LookingGlass() as what:
    print('Mathias, Alice, and Nowdrop')
    print(what)

pordwoN dna ,ecilA ,saihtaM
MAERCECI


In [11]:
what

'ICECREAM'

In [12]:
manager = LookingGlass()
manager

In [17]:
monster = manager.__enter__()
monster == 'ICECREAM'

True

In [18]:
monster

'ICECREAM'

In [19]:
manager

In [20]:
manager.__exit__(None, None, None)
monster

'ICECREAM'

In [2]:
# using @contextmanager saves us from writting __enter__/__exit__
# __enter__ runs before yield, and __exit__ after yield
# yield will be bounded to as
import contextlib
import sys

@contextlib.contextmanager
def looking_glass():
    original_write = sys.stdout.write

    def reverse_write(text):
        original_write(text[::-1])

    sys.stdout.write = reverse_write
    try:
        yield 'ICECREAM'
    finally:
        sys.stdout.write = original_write

In [3]:
with looking_glass() as what:
    print('Mathias, Vanilla and Snowcone')
    print(what)

enocwonS dna allinaV ,saihtaM
MAERCECI


In [4]:
what

'ICECREAM'

In [13]:
import contextlib
import sys

@contextlib.contextmanager
def looking_glass():
    original_write = sys.stdout.write

    def reverse_write(text):
        original_write(text[::-1])

    sys.stdout.write = reverse_write
    msg = ''
    try:
        yield 'ICECREAM'
    except ZeroDivisionError:
        msg = 'Please DO NOT divide by zero!'
    finally:
        sys.stdout.write = original_write
        if msg:
            print(msg)

In [16]:
@looking_glass()
def verse():
    print('The future is now')

verse()

won si erutuf ehT


In [22]:
def mod(m, n):
    return m - (m // n * n)

def gcd(m, n):
    if n == 0:
        return m
    else:
        return gcd(n, mod(m, n))

print(gcd(18, 45))

9


In [25]:
from collections import ChainMap
from typing import Any, TypeAlias

Symbol: TypeAlias = str

class Environment(ChainMap[Symbol, Any]):
    def change(self, key: Symbol, value: Any) -> None:
        for map in self.maps:
            if key in map:
                map[key] = value
                return
        raise KeyError(key)

In [26]:
inner_env = {'a': 2}
outer_env = {'a': 0, 'b': 1}
env = Environment(inner_env, outer_env)
env['a']

2

In [27]:
env['a'] = 111
env['c'] = 222
env

Environment({'a': 111, 'c': 222}, {'a': 0, 'b': 1})

In [28]:
env.change('b', 333)
env

Environment({'a': 111, 'c': 222}, {'a': 0, 'b': 333})

In [30]:
import math

def standard_env() -> Environment:
    env = Environment()
    env.update(vars(math)) # sin, cos, sqrt, etc

In [33]:
# just used as an example to parse Scheme, and create an AST
Atom: TypeAlias = float | int | Symbol
Expression: TypeAlias = Atom | list

KEYWORDS = ['quote', 'if', 'lambda', 'define', 'set!']

def evaluate(exp: Expression, env: Environment) -> Any:
    match exp:
        case int(x) | float(x):
            return x
        case Symbol(var):
            return env[var]
        case ['quote', x]:
            return x
        case ['if', test, consequence, alternative]:
            if evaluate(test, env):
                return evaluate(consequence, env)
            else:
                return evaluate(alternative, env)
        case ['lambda', [*parms], *body] if body:
            return Procedure(parms, body, env)
        case ['define', Symbol(name), value_exp]:
            env[name] = evaluate(value_exp, env)
        case ['define', [Symbol(name), *parms], *body] if body:
            env[name] = Procedure(parms, body, env)
        case ['set!', Symbol(name), value_exp]:
            env.change(name, evaluate(value_exp, env))
        case [func_exp, *args] if func_exp not in KEYWORDS:
            proc = evaluate(func_exp, env)
            values = [evaluate(arg, env) for arg in args]
            return proc(*values)
        case _:
            raise SyntaxError(listpstr(exp))

In [35]:
# (if (< x 0) 0 x)
# case ['if', test, consequence, alternative]

In [36]:
# (lambda (a b) (/ (+ a b) 2))
# case ['lambda', [*parms], *body]

In [37]:
# (quote (99 bottles of beer))
# case ['quote', x]:

In [38]:
# (define half (/ 1 2))
# case ['define', Symbol(name), value_exp]

In [39]:
# (define (average a b) (/ (+ a b) 2))
# case ['define', [Symbol(name), *parms], *body]

In [40]:
# (set! n (+ n 1))
# case ['set!', Symbol(name), value_exp]:

In [41]:
"""
Our averager function in Scheme:
(define (make-averager)
    (define count 0)
    (define total 0)
    (lambda (new-value)
        (set! count (+ count 1))
        (set! total (+ total new-value))
        (/ total count)
    )
)

(define avg (make-averager))
(avg 10)
(avg 11)
(avg 15)
"""

'\nOur averager function in Scheme:\n(define (make-averager)\n    (define count 0)\n    (define total 0)\n    (lambda (new-value)\n        (set! count (+ count 1))\n        (set! total (+ total new-value))\n        (/ total count)\n    )\n)\n\n(define avg (make-averager))\n(avg 10)\n(avg 11)\n(avg 15)\n'

In [42]:
# (gcd (* 2 105) 84)
# case [func_exp, *args]

In [43]:
class Procedure:
    def __init__(self, parms: list[Symbol], body: list[Expression], env: Environment):
        self.parms = parns
        self.body = body
        self.env = env

    def __call__(self, *args: Expression) -> Any:
        local_env = dict(zip(parms, args))
        env = Environment(local_env, self.env)
        for exp in self.body:
            result = evaluate(exp, env)
        return result

In [44]:
# In a case clause the | operator has special meaning it does not trigger __or__
# (λ (a b) (/ (+ a b) 2) )
# case ['lambda' | 'λ', [*parms], *body]

In [64]:
# else will run only if and when the for loop runs to completion (not if the for is aborted)
class FlavorItem:
    def __init__(self, flavor):
        self.flavor = flavor
        
my_list = [
    FlavorItem("chocolate"),
    FlavorItem("vanilla"),
    FlavorItem("strawberry"),
    FlavorItem("bananas"),
]
for item in my_list:
    if item.flavor == 'banana':
       break
else:
   raise ValueError('No banana flavor found!')

ValueError: No banana flavor found!

In [65]:
# try block should only have statements that may generate the expected exceptions
"""
BAD!
try:
    dangerous_call()
    after_call()
except OSError:
    log('OSError...')

GOOD!
try:
    dangerous_call()
except OSError:
    log('OSError...')
else:
    after_call()
"""

"\nBAD!\ntry:\n    dangerous_call()\n    after_call()\nexcept OSError:\n    log('OSError...')\n\nGOOD!\ntry:\n    dangerous_call()\nexcept OSError:\n    log('OSError...')\nelse:\n    after_call()\n"

In [66]:
"""
Coding styles

EAFP:
Easier to ask for forgiveness than permission.

LBYL:
Look before you leap.
"""

'\nCoding styles\n\nEAFP:\nEasier to ask for forgiveness than permission.\n\nLBYL:\nLook before you leap.\n'

In [67]:
"""
non-tail factorial
def factorial(n):
    if n < 2:
        return 1
    return n * factorial(n - 1)

tail factorial
def factorial_tc(n, product=1):
    if n < 1:
        return product
    return factorial_tc(n - 1, product * n)
"""

'\nnon-tail factorial\ndef factorial(n):\n    if n < 2:\n        return 1\n    return n * factorial(n - 1)\n\ntail factorial\ndef factorial_tc(n, product=1):\n    if n < 1:\n        return product\n    return factorial_tc(n - 1, product * n)\n'